# Using capellambse with Jinja2

Welcome to the py-capellambse jinja2 templating showcase. When using capella
for systems engineering you might want to generate documentation for your model
you can use M2DOC, one of capella's [addons] which we found too challenging or
you can use Jinja2 a richful templating language with high degree of freedom.

This notebook will introduce you into writing jinja2 templates where you'll plant model
information and diagrams. Additonally we'll give a side-note on how to handle
unique identifiers professionally with PVMT and some hints onto how to use jinja
in a professional manner which could give you the option onto developping an
automated document generation system.

With Jinja2 you are able to generate any text-based format(HTML, XML, CSV, LaTex,...)
but during this tutorial we will only generate .html files. The jinja2 syntax is
inspired by python. Check out their [docs]!

[docs]: https://jinja2docs.readthedocs.io/en/stable/
[addons]: https://www.eclipse.org/capella/addons.html

Below code loads the needed libraries and instantiates a test model:

In [1]:
import jinja2
import capellambse

from IPython.core.display import HTML

path_to_model = "../../../tests/data/melodymodel/5_0/Melody Model Test.aird"
model = capellambse.MelodyModel(path_to_model, jupyter_untrusted=True)

Cannot load PVMT extension: ValueError: Provided model does not have a PropertyValuePkg
Property values are not available in this model


In the following we want to make a template to document all modelled actors from
the logical layer. Therefore we define a template string where we iterate over
all actors and plant the name, uuid and description into it.

*Hint: Make sure that you know of capella's [metamodel] as we are implementing the
capellambse.layers as close as possible to it while being as efficiently and pythonic
we can be currently. This knowledge can shorten used statements in the template immensely!*

[metamodel]: https://dsd-dbs.github.io/py-capellambse/start/intro-to-api.html

In [2]:
env = jinja2.Environment()
templ = """
<h1>Actor definitions</h1>
{% for actor in model.la.all_components.by_is_actor(True) %}
    <h2>{{ actor.name }}</h2>
    <h3>Actor definition</h3>
    <p>UUID: {{ actor.uuid }}</p>
    <p>{{ actor.description }}</p>
{% endfor %}
"""

HTML(env.from_string(templ).render(model=model))

As an extra: We don't use the UUID from capella in our documents. If you still
need an identifier, in the following it's explained how we are doing it:

With the capella PVMT, one of the various capella [addons], you can make property
value groups and then set arbitrary values with these. Capellambse is able to recognize
the pvmt extension and gives read and write access. In our workflows we are maintaining
an ID database for all model elements. If that is done you can access pvmt attributes
like:

```html
{{ ... }}
<p>ID: {{ actor.pvmt["Group.Identification.MY MODEL ID"] }}</p>
{{ ... }}
```

For a PVMT showcase look into [TODO: pvmt-showcase notebook].

## Filters and object manipulation

Now to step up our templating-game, we'll bring in more complexity.
We want a template that documents functional context of all actors. For that
iff the actor has a non-empty functions attribute we make a table with columns
for function's uid, name, description and `FunctionalExchange`s.

We can define variables in the template via the set statement. Furthermore
the builtin jinja [filters] already give much power for object manipulation during
rendering. Here we use map(.) to create `FunctionalExchange` iterators and sum these
up into one large list that stores all `FunctionalExchange`s that have an an actor
as either source or target. In the table for-loop we then filter on this lookup
container and set outgoing and incoming `FunctionalExchange`s that we need for
the last column.

The jupyter environment is great for writing templates b/c you can investigate
possible attributes of objects right away in another cell.

*Hint: You can define custom filter-functions and add them to the Environment.filters.*

[filters]: https://jinja.palletsprojects.com/en/3.0.x/templates/#builtin-filters

In [3]:
templ1 = """
<h1>Actor definitions</h1>
{% set fexs = model.la.actor_exchanges | map(attribute="func_exchanges") | map('list')
    | sum(start=[]) | list %}
{% for actor in model.la.all_actors %}
    <h2>{{ actor.name }}</h2>
    <h3>Actor definition</h3>
    <p>{{ actor.description }}</p>
    <h3>Actor functions</h3>
    {% if actor.functions %}
        <p>The table below identifies functions of {{ actor.name }}.</p>
        <table>
            <thead>
                <tr>
                    <th>ID, Function</th>
                    <th>Description</th>
                    <th>Involved Subsystems</th>
                </tr>
            </thead>
            <tbody>
            {% for fnc in actor.functions %}
                {% set ports = fnc.inputs + fnc.outputs %}
                {% set outs = (fexs | selectattr("source_port", "in", ports) | list) 
                    | map(attribute="target_port.owner.owner.name") | list %}
                {% set ins = (fexs | selectattr("target_port", "in", ports) | list)
                    | map(attribute="source_port.owner.owner.name") | list %}
                {% set subs = (ins + outs) | unique | list %}
                <tr>
                    <td colspan="1">
                        <p>{{ fnc.uuid }}</p>
                        <p>{{ fnc.name }}</p>
                    </td>
                    <td colspan="1">{{ fnc.description }}</td>
                    <td colspan="1">{{ subs | join(', ') }}</td>
                </tr>
            {% endfor %}
            </tbody>
        </table>
        <p style="text-align: center;"><strong>Functions of {{ actor.name }}</strong></p>
    {% else %}
        <p style="text-align: left;">No actor functions were identified.</p>
    {% endif %}
{% endfor %}
"""
HTML(env.from_string(templ1).render(model=model))

"ID, Function",Description,Involved Subsystems
f708bc29-d69f-42a0-90cc-11fc01054cd0 manage the school,None,
beaf5ba4-8fa9-4342-911f-0266bb29be45 advise Harry,None,
"ID, Function",Description,Involved Subsystems
a7acb298-d14b-4707-a419-fea272434541 teach Potions,None,Hogwarts
4a2a7f3c-d223-4d44-94a7-50dd2906a70c maintain a layer of defense for the Sorcerer's Stone,None,
"ID, Function",Description,Involved Subsystems
aa9931e3-116c-461e-8215-6b9fdbdd4a1b kill He Who Must Not Be Named,None,School
"ID, Function",Description,Involved Subsystems
c1a42acc-1f53-42bb-8404-77a5c08c414b assist Harry,None,
edbd1ad4-31c0-4d53-b856-3ffa60e0e99b break school rules,None,Whomping Willow


## Beautiful SVG diagrams

Finally we will render a template that displays a diagram. There are many ways
to do this and with jinja2 you have full control. We like our figures inside
tables such that a caption can be displayed

In [4]:
figure_markup = (
    "<table>"
    "<caption><strong>Figure {fig_id}:</strong> {caption}</caption>"
    "<tr><td>{diagram}</td></tr>"
    "</table>"
)
templ = """<h1>Wizzard Education</h1>
{% set hogwarts = model.la.all_components.by_name("Hogwarts") %}
<h2>{{ hogwarts.name }}</h2>
{{ hogwarts.description }}
{{ figure_table }}"""
diagram = model.diagrams.by_name("[LAB] Wizzard Education")
fig_table = figure_markup.format(
    fig_id=1,
    caption=f"<i>{diagram.name}</i>",
    diagram=diagram.as_svg
)
rendered = env.from_string(templ).render(model=model, figure_table=fig_table)
HTML(rendered)

Unknown global filter 'hide.sequencing.information.filter'
Unknown global filter 'ModelExtensionFilter'
Unknown global filter 'hide.simplified.diagram.based.component.exchanges.filter'
Unknown global filter 'hide.simplified.oriented.grouped.component.exchanges.filter'
Unknown global filter 'hide.simplified.group.of.component.exchanges.filter'


*Hint: Take notice of the [jinja.Environment]. Instead of handing the figure_table-markup
over in the rendering call you could also set an insert_figure_as_table function,
which you ideally defined before, in the environment globals or you can define
[macros] right in the template. These tools can automate repetitive content placement.*

[jinja.Environment]: https://jinja.palletsprojects.com/en/3.0.x/api/#jinja2.Environment
[macros]: https://jinja.palletsprojects.com/en/3.0.x/templates/#macros

## Template-inheritance

The last cell will present a routine where a full .html document is generated.
Earlier rendered content were HTML-fragments to be precise. On top you can see
a showcase on jinja's [template-inheritance] functionality. The special DictLoader
gives support for finding the base template called "template.html". This was just
needed because we are dealing with content in memory and didn't create template.html
in our FileSystem before. Per default jinja is using the FileSystemLoader when
creating an Environment. It's not a bad idea to check the [Loaders] they have, if
you want to understand how template loading is working and/or plan on developing
a pipeline system for document distribution.

[template-inheritance]: https://jinja.palletsprojects.com/en/3.0.x/templates/#template-inheritance
[Loaders]: https://jinja.palletsprojects.com/en/3.0.x/api/#loaders

In [5]:
fig_templ = "".join(
    (
        '{% extends "template.html" %}',
        "{% block content %}",
        templ,
        "{% endblock %}",
    )
)
final_templ = """
<!DOCTYPE html>
<html>

<head>
    <style>
        caption {
            caption-side: bottom;
            border-top: 1px solid silver;
        }
    </style>
</head>

<body>
    {% block content %}
    {% endblock %}
</body>
</html>
"""

env = jinja2.Environment(loader=jinja2.DictLoader({"template.html": final_templ}))
rendered = env.from_string(fig_templ).render(model=model, figure_table=fig_table)
HTML(rendered)